## Creating the main (historic) dataframe

In this notebook, we join the FEMA Disasters data, Bureau of Labor Statistics' employment statistics by state, and the Bureau of Labor Statistics' occupational employment statistics by state. Each of these three datasets has been initially cleaned in other notebooks.

### Imports and Useful Functions:

In [1]:
import pandas as pd

In [2]:
def null_cleaning(df):
    null_cols = []
    problem_cols = []
    
    # looping through the columns
    for i in range(df.shape[1]):
        # counting the number of null vals in the column
        num_nulls = df.isnull().sum()[i]
        # if there are more than 0 null values, add the column to our list, and see what % of the data it is
        if num_nulls != 0:
            col_name = df.columns[i]
            percent_of_data = round(num_nulls/df.shape[0], 3)
            null_cols.append([col_name, num_nulls, percent_of_data])
            # if this column is over 20% nulls, then mark it as a problem column
            if percent_of_data >= 0.2:
                problem_cols.append(col_name)
                
    # what happens if we drop the problem columns?
    df_no_prob_cols = df.drop(columns = problem_cols)
    prob_cols_percent = df_no_prob_cols.shape[1] / df.shape[1]
    
    #what happens is we drop the problem rows too?
    df_no_nulls = df_no_prob_cols.dropna()
    prob_rows_percent = df_no_nulls.shape[0]/df_no_prob_cols.shape[0]
    
    print("The problem columns are: ", problem_cols)
    print("After dropping the problem columns, you are left with ", prob_cols_percent, "% of your columns.")
    print("After dropping the problem columns, and then dropping all rows containing nulls,\n you are left with ", 
          prob_rows_percent, "% of your rows.")
    return null_cols


### FEMA Disasters list

We will join the other data onto this one, so that each row (eventually) represents one disaster event, in one state, and its impact on one specific occupation.

In [33]:
disasters = pd.read_csv('../data/disasters_clean.csv')

In [35]:
disasters.drop(columns = ['Unnamed: 0'], inplace = True)

In [36]:
disasters.head()

,disasterNumber,state,incidentType,year,month
0,495,MI,Severe Storm(s),1976,3
1,494,NY,Severe Ice Storm,1976,3
2,496,WI,Flood,1976,3
3,497,OK,Tornado,1976,4
4,498,AR,Tornado,1976,4


Our other data only goes as far back as 1997, so we restrict to 1997 on:

In [37]:
disaster_df = disasters[(disasters['year'] >= 1997) & (disasters['year'] < 2020)]

In [39]:
disaster_df.shape

(2675, 5)

In [40]:
disaster_df.dtypes

disasterNumber     int64
state             object
incidentType      object
year               int64
month              int64
dtype: object

The following lines of code with the dataframe "d" process the declaration date as a datetime object, and save the date one month before adn two months after each event so that wage change can be (eventually) calculated across that time period

In [41]:
d = disaster_df.astype({'year':'str','month':'str'})

In [43]:
d['y_m'] = d['year']+d['month']

In [44]:
d['y_m'] = pd.to_datetime(d['y_m'],format = '%Y%m')

In [45]:
d.dtypes

disasterNumber             int64
state                     object
incidentType              object
year                      object
month                     object
y_m               datetime64[ns]
dtype: object

In [47]:
d['date_month_before'] = d['y_m'] - pd.DateOffset(months=1)

In [49]:
d['date_two_months_after'] = d['y_m'] + pd.DateOffset(months=2)

In [51]:
d['month_before'] = [t.month for t in d['date_month_before']]

In [52]:
d['year_before'] = [t.year for t in d['date_month_before']]

In [53]:
d['two_months_after'] = [t.month for t in d['date_two_months_after']]

In [54]:
d['year_two_months_after'] = [t.year for t in d['date_two_months_after']]

In [55]:
d.head()

,disasterNumber,state,incidentType,year,month,y_m,date_month_before,date_two_months_after,month_before,year_before,two_months_after,year_two_months_after
392,1190,NE,Severe Storm(s),1997,11,1997-11-01,1997-10-01,1998-01-01,10,1997,1,1998
399,1173,SD,Flood,1997,4,1997-04-01,1997-03-01,1997-06-01,3,1997,6,1997
401,1174,ND,Flood,1997,4,1997-04-01,1997-03-01,1997-06-01,3,1997,6,1997
408,1157,ND,Severe Storm(s),1997,1,1997-01-01,1996-12-01,1997-03-01,12,1996,3,1997
421,1208,AL,Severe Storm(s),1998,3,1998-03-01,1998-02-01,1998-05-01,2,1998,5,1998


In [56]:
d.drop(columns = ['y_m','date_month_before','date_two_months_after'], inplace = True)

In [57]:
d.head()

,disasterNumber,state,incidentType,year,month,month_before,year_before,two_months_after,year_two_months_after
392,1190,NE,Severe Storm(s),1997,11,10,1997,1,1998
399,1173,SD,Flood,1997,4,3,1997,6,1997
401,1174,ND,Flood,1997,4,3,1997,6,1997
408,1157,ND,Severe Storm(s),1997,1,12,1996,3,1997
421,1208,AL,Severe Storm(s),1998,3,2,1998,5,1998


In [68]:
d = d.astype({'year':'float64','month':'float64'})

### Joining occupational (job) statistics:

In [58]:
jobs_df = pd.read_csv('../data/occupation_99_to_19.csv')

In [59]:
jobs_df.head()

,st,occ_code,occ_title,tot_emp,h_mean,a_mean,year
0,AL,10000,Managerial and Administrative Occupations,NaN,NaN,NaN,NaN
1,AL,13000,Staff and Administrative Specialty Managerial ...,**,18.04,37530,1997.0
2,AL,13002,Financial Managers,8140,24.23,50400,1997.0
3,AL,13005,"Personnel, Training, and Labor Relations Managers",2760,20.84,43350,1997.0
4,AL,13008,Purchasing Managers,2370,19.07,39660,1997.0


Checking null values and what percentage of the data they are: 

In [60]:
null_cleaning(jobs_df)

The problem columns are:  []
After dropping the problem columns, you are left with  1.0 % of your columns.
After dropping the problem columns, and then dropping all rows containing nulls,
 you are left with  0.9990797288365021 % of your rows.


[['tot_emp', 756, 0.001],
 ['h_mean', 756, 0.001],
 ['a_mean', 756, 0.001],
 ['year', 756, 0.001]]

Nulls make up less than 1% of the occupational data, so we drop them: 

In [61]:
jobs_df.dropna(inplace = True)

In [62]:
# renaming so that we can join on the state and year columns
jobs_df.rename(columns = {'st':'state'}, inplace = True)

In [69]:
df = d.merge(jobs_df, on = ['state','year'])

In [71]:
df.shape

(1780516, 14)

In [72]:
df.columns

Index(['disasterNumber', 'state', 'incidentType', 'year', 'month',
       'month_before', 'year_before', 'two_months_after',
       'year_two_months_after', 'occ_code', 'occ_title', 'tot_emp', 'h_mean',
       'a_mean'],
      dtype='object')

In [ ]:
# saving this as data in case we want to play around with it
df.to_csv('../data/disasters_occupations_clean.csv', index = False)

### Joining unemployment rates: 

In [89]:
unemployment_df = pd.read_csv('../data/unemployment_by_state_1976.csv')

In [90]:
unemployment_df.head()

,FIPS_Code,state_and_area,period_year,period_month,area_population,civ_lf_total,civ_lf_total_%,civ_lf_employed,civ_lf_employed_%,civ_lf_unemployed,civ_lf_unemployed_%
0,1,Alabama,1976,1,2605000,1492409,57.3,1392154,53.4,100255,6.7
1,2,Alaska,1976,1,232000,159154,68.6,147809,63.7,11345,7.1
2,4,Arizona,1976,1,1621000,972413,60.0,872738,53.8,99675,10.3
3,5,Arkansas,1976,1,1536000,882835,57.5,817756,53.2,65079,7.4
4,6,California,1976,1,15621000,9781720,62.6,8892663,56.9,889057,9.1


In [91]:
unemployment_df.rename(columns = {'state_and_area':'state',
                                  'period_year':'year',
                                  'period_month':'month'}, 
                       inplace = True)

In [92]:
unemployment_df.columns

Index(['FIPS_Code', 'state', 'year', 'month', 'area_population',
       'civ_lf_total', 'civ_lf_total_%', 'civ_lf_employed',
       'civ_lf_employed_%', 'civ_lf_unemployed', 'civ_lf_unemployed_%'],
      dtype='object')

In [93]:
features = ['state', 'year', 'month', 'area_population','civ_lf_employed_%']
unemployment_df = unemployment_df[features].copy()

In [94]:
unemployment_df.head()

,state,year,month,area_population,civ_lf_employed_%
0,Alabama,1976,1,2605000,53.4
1,Alaska,1976,1,232000,63.7
2,Arizona,1976,1,1621000,53.8
3,Arkansas,1976,1,1536000,53.2
4,California,1976,1,15621000,56.9


The states are recorded by their abbreviations in all other dataframes except the employment stats data, so we must change that:

In [95]:
# I got this from http://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/

states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [96]:
# swapping the key-value pairs:

new_states_dict = dict([(value, key) for key, value in states_dict.items()])
new_states_dict

{'Alaska': 'AK',
 'Alabama': 'AL',
 'Arkansas': 'AR',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'District of Columbia': 'DC',
 'Delaware': 'DE',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Iowa': 'IA',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Massachusetts': 'MA',
 'Maryland': 'MD',
 'Maine': 'ME',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Missouri': 'MO',
 'Northern Mariana Islands': 'MP',
 'Mississippi': 'MS',
 'Montana': 'MT',
 'National': 'NA',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Nebraska': 'NE',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'Nevada': 'NV',
 'New York': 'NY',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 

In [97]:
unemployment_df['state'] = unemployment_df['state'].map(new_states_dict)

In [98]:
unemployment_df.head()

,state,year,month,area_population,civ_lf_employed_%
0,AL,1976,1,2605000,53.4
1,AK,1976,1,232000,63.7
2,AZ,1976,1,1621000,53.8
3,AR,1976,1,1536000,53.2
4,CA,1976,1,15621000,56.9


In [100]:
# cutting to 1996 and on:

unemployment_df = unemployment_df[(unemployment_df['year'] >= 1996)].copy()

In [101]:
unemployment_df.head()

,state,year,month,area_population,civ_lf_employed_%
12720,AL,1996,1,3270552,59.8
12721,AK,1996,1,418371,67.8
12722,AZ,1996,1,3412217,62.6
12723,AR,1996,1,1927059,61.4
12724,CA,1996,1,23347682,60.4


In [86]:
# checking null values:
null_cleaning(unemployment_df)

The problem columns are:  []
After dropping the problem columns, you are left with  1.0 % of your columns.
After dropping the problem columns, and then dropping all rows containing nulls,
 you are left with  0.9622641509433962 % of your rows.


[['state', 552, 0.038]]

Now we merge the employment stats to the disasters and occupations data along year, state, and month. We do this three times, first on the declaration date of the disaster, again on the date the month before each disaster, and finally on the date two months after each disaster. This way we can calculate wage change across this time period:

In [104]:
df1 = df.merge(unemployment_df, on = ['state','year','month'])

In [105]:
df1.head()

,disasterNumber,state,incidentType,year,month,month_before,year_before,two_months_after,year_two_months_after,occ_code,occ_title,tot_emp,h_mean,a_mean,area_population,civ_lf_employed_%
0,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13000,Staff and Administrative Specialty Managerial ...,**,16.9,35150,1261157,71.5
1,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13002,Financial Managers,3730,24.50,50960,1261157,71.5
2,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13005,"Personnel, Training, and Labor Relations Managers",1420,21.41,44540,1261157,71.5
3,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13008,Purchasing Managers,1300,18.76,39030,1261157,71.5
4,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13011,"Marketing, Advertising, and Public Relations M...",3380,23.06,47970,1261157,71.5


In [106]:
employment_before_df = unemployment_df.copy()

In [107]:
employment_before_df.rename(columns = {'civ_lf_employed_%':'employment_rate_before',
                                       'month':'month_before',
                                       'year':'year_before'},
                           inplace= True)

In [108]:
employment_after_df = unemployment_df.copy()

In [109]:
employment_after_df.rename(columns = {'civ_lf_employed_%':'employment_rate_after',
                                     'month':'two_months_after',
                                     'year':'year_two_months_after'},
                          inplace = True)

In [112]:
df1.rename(columns = {'civ_lf_employed_%':'employment_rate_during'}, inplace = True)

In [114]:
df2 = df1.merge(employment_before_df, on = ['state','year_before','month_before'])

In [116]:
df3 = df2.merge(employment_after_df, on = ['state','year_two_months_after','two_months_after'])

In [118]:
df3.drop(columns = ['area_population_x'], inplace = True)

In [119]:
df3.rename(columns = {'area_population_y':'population_before','area_population':'population_after'})

,disasterNumber,state,incidentType,year,month,month_before,year_before,two_months_after,year_two_months_after,occ_code,occ_title,tot_emp,h_mean,a_mean,employment_rate_during,population_before,employment_rate_before,population_after,employment_rate_after
0,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13000,Staff and Administrative Specialty Managerial ...,**,16.9,35150,71.5,1260913,71.5,1262644,71.6
1,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13002,Financial Managers,3730,24.50,50960,71.5,1260913,71.5,1262644,71.6
2,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13005,"Personnel, Training, and Labor Relations Managers",1420,21.41,44540,71.5,1260913,71.5,1262644,71.6
3,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13008,Purchasing Managers,1300,18.76,39030,71.5,1260913,71.5,1262644,71.6
4,1190,NE,Severe Storm(s),1997.0,11.0,10,1997,1,1998,13011,"Marketing, Advertising, and Public Relations M...",3380,23.06,47970,71.5,1260913,71.5,1262644,71.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764431,5053,TX,Fire,2014.0,5.0,4,2014,7,2014,53-7072,"Pump Operators, Except Wellhead Pumpers",4110,20.87,43400,61.6,19999657,61.6,20101104,61.6
1764432,5053,TX,Fire,2014.0,5.0,4,2014,7,2014,53-7073,Wellhead Pumpers,2900,23.55,48990,61.6,19999657,61.6,20101104,61.6
1764433,5053,TX,Fire,2014.0,5.0,4,2014,7,2014,53-7081,Refuse and Recyclable Material Collectors,7320,14.09,29300,61.6,19999657,61.6,20101104,61.6
1764434,5053,TX,Fire,2014.0,5.0,4,2014,7,2014,53-7121,"Tank Car, Truck, and Ship Loaders",2680,19.77,41130,61.6,19999657,61.6,20101104,61.6


In [120]:
df4 = df3.drop(columns = ['month_before','year_before','two_months_after','year_two_months_after'])

In [122]:
df4.rename(columns = {'area_population_y':'population_before','area_population':'population_after'},inplace = True)

In [123]:
df4.head()

,disasterNumber,state,incidentType,year,month,occ_code,occ_title,tot_emp,h_mean,a_mean,employment_rate_during,population_before,employment_rate_before,population_after,employment_rate_after
0,1190,NE,Severe Storm(s),1997.0,11.0,13000,Staff and Administrative Specialty Managerial ...,**,16.9,35150,71.5,1260913,71.5,1262644,71.6
1,1190,NE,Severe Storm(s),1997.0,11.0,13002,Financial Managers,3730,24.50,50960,71.5,1260913,71.5,1262644,71.6
2,1190,NE,Severe Storm(s),1997.0,11.0,13005,"Personnel, Training, and Labor Relations Managers",1420,21.41,44540,71.5,1260913,71.5,1262644,71.6
3,1190,NE,Severe Storm(s),1997.0,11.0,13008,Purchasing Managers,1300,18.76,39030,71.5,1260913,71.5,1262644,71.6
4,1190,NE,Severe Storm(s),1997.0,11.0,13011,"Marketing, Advertising, and Public Relations M...",3380,23.06,47970,71.5,1260913,71.5,1262644,71.6


In [124]:
df4['employment_rate_change'] = df4['employment_rate_before']-df4['employment_rate_after']

In [126]:
df4 = df4[(df4['h_mean'] != '*') & (df4['h_mean']!= '#') ]

In [127]:
df4 = df4[df4['tot_emp'] != '**'].copy()

In [128]:
df4 = df4.astype({'h_mean':'float64','tot_emp':'float64'}).copy()

Here we calculate wage loss:

(percent change in state employment rate)x(hourly average wage for that job in that state at that time)x(40 hrs per week)x(12 weeks)x(number of people employed in that job type in that state)

In [130]:
df4['wage_loss'] = (df4['employment_rate_change']/100)*df4['h_mean']*40*12*df4['tot_emp']

In [132]:
df4.rename(columns = {'wage_loss':'wage_change'},inplace = True)

In [135]:
df4.head()

,disasterNumber,state,incidentType,year,month,occ_code,occ_title,tot_emp,h_mean,a_mean,employment_rate_during,employment_rate_before,employment_rate_after,employment_rate_change,wage_change
1,1190,NE,Severe Storm(s),1997.0,11.0,13002,Financial Managers,3730.0,24.50,50960,71.5,71.5,71.6,-0.1,-43864.800
2,1190,NE,Severe Storm(s),1997.0,11.0,13005,"Personnel, Training, and Labor Relations Managers",1420.0,21.41,44540,71.5,71.5,71.6,-0.1,-14593.056
3,1190,NE,Severe Storm(s),1997.0,11.0,13008,Purchasing Managers,1300.0,18.76,39030,71.5,71.5,71.6,-0.1,-11706.240
4,1190,NE,Severe Storm(s),1997.0,11.0,13011,"Marketing, Advertising, and Public Relations M...",3380.0,23.06,47970,71.5,71.5,71.6,-0.1,-37412.544
5,1190,NE,Severe Storm(s),1997.0,11.0,13014,Administrative Services Managers,2060.0,19.43,40410,71.5,71.5,71.6,-0.1,-19212.384


In [134]:
df4.drop(columns = ['population_before','population_after'],inplace = True)

In [136]:
df4.shape

(1566063, 15)

All merged and ready to go! Saving as a csv file:

In [137]:
df4.to_csv('../data/main_df.csv', index = False)

### Creating Data Dictionary: 


|Feature|Type|Dataset|Description|
|---|---|---|---|
|**disasterNumber**|*integer*|main_df|ID number associated to each disaster event.| 
|**state**|*object*|main_df|State where disaster occurred.|
|**incidentType**|*object*|main_df|Type of disaster (tornado, fire, hurricane, etc.).|
|**year**|*float*|main_df|Year disaster declared.|
|**month**|*float*|main_df|Month dsaster declared.|
|**occ_code**|*object*|main_df|Occupation code (first two digits indicate industry type).|
|**occ_title**|*object*|main_df|Occupation title.|
|**tot_emp**|*integer*|main_df|Total number of people employed in occupation in state at time of disaster.|
|**h_mean**|*integer*|main_df|Hourly average wage for occupation in state.|
|**a_mean**|*integer*|main_df|Annual average wage for occupation in state.|
|**employment_rate_during**|*integer*|main_df|State employment rate at time of disaster.|
|**employment_rate_before**|*integer*|main_df|State employment rate at one month before time of disaster.|
|**employment_rate_after**|*integer*|main_df|State employment rate two months after time of disaster.|
|**wage_change**|*integer*|main_df|Wage change across time one month before disaster and two months after. Total wages gained or lost from all people working in a particular occupation in each state at time of disaster.|